In [ ]:
import sqlite3
import pandas as pd

# Loading the cleaned CSV from GitHub
url = "https://raw.githubusercontent.com/pakizaim7/spacex-capstone/main/notebooks/spacex_launch_data_cleaned.csv"
df = pd.read_csv(url)

# Connecting to SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Writing the DataFrame to a table called 'launches'
df.to_sql("launches", conn, index=False, if_exists="replace")

# Checking the table
pd.read_sql("SELECT * FROM launches LIMIT 5;", conn)


,flight_number,name,launch_year,success,class,rocket,payloads,launchpad
0,1,FalconSat,2006,0,0,5e9d0d95eda69955f709d1eb,['5eb0e4b5b6c3bb0006eeb1e1'],5e9e4502f5090995de566f86
1,2,DemoSat,2007,0,0,5e9d0d95eda69955f709d1eb,['5eb0e4b6b6c3bb0006eeb1e2'],5e9e4502f5090995de566f86
2,3,Trailblazer,2008,0,0,5e9d0d95eda69955f709d1eb,"['5eb0e4b6b6c3bb0006eeb1e3', '5eb0e4b6b6c3bb00...",5e9e4502f5090995de566f86
3,4,RatSat,2008,1,1,5e9d0d95eda69955f709d1eb,['5eb0e4b7b6c3bb0006eeb1e5'],5e9e4502f5090995de566f86
4,5,RazakSat,2009,1,1,5e9d0d95eda69955f709d1eb,['5eb0e4b7b6c3bb0006eeb1e6'],5e9e4502f5090995de566f86


In [ ]:
# For demonstration, creating a dummy numeric payload_mass column
import numpy as np

# Random payload mass between 1000 and 10000 kg for each launch
np.random.seed(42)
df['payload_mass'] = np.random.randint(1000, 10001, size=len(df))

# Saving updated DataFrame to SQLite again
df.to_sql("launches", conn, index=False, if_exists="replace")


187

In [ ]:
# Getting column names from SQLite table
query = "PRAGMA table_info(launches);"
pd.read_sql(query, conn)


,cid,name,type,notnull,dflt_value,pk
0,0,flight_number,INTEGER,0,None,0
1,1,name,TEXT,0,None,0
2,2,launch_year,INTEGER,0,None,0
3,3,success,INTEGER,0,None,0
4,4,class,INTEGER,0,None,0
5,5,rocket,TEXT,0,None,0
6,6,payloads,TEXT,0,None,0
7,7,launchpad,TEXT,0,None,0
8,8,payload_mass,INTEGER,0,None,0


Slide 24: All Lunch Site names

In [ ]:
query = "SELECT DISTINCT launchpad FROM launches;"
pd.read_sql(query, conn)


,launchpad
0,5e9e4502f5090995de566f86
1,5e9e4501f509094ba4566f84
2,5e9e4502f509092b78566f87
3,5e9e4502f509094188566f88


Slide 25: Launch sites begining with 'CCA'

In [ ]:
query = "SELECT launchpad FROM launches WHERE launchpad LIKE 'CCA%' LIMIT 5;"
pd.read_sql(query, conn)


,launchpad


Slide 26: Total Payload Mass

In [ ]:
query = "SELECT SUM(payload_mass) AS total_payload FROM launches;"
pd.read_sql(query, conn)


,total_payload
0,1075174


In [ ]:
df['rocket'].unique()


array(['5e9d0d95eda69955f709d1eb', '5e9d0d95eda69973a809d1ec',
       '5e9d0d95eda69974db09d1ed'], dtype=object)

Slide 27: Average Payload Mass by F9 v1.1

In [ ]:
query = """
SELECT AVG(payload_mass) AS avg_payload
FROM launches
WHERE rocket LIKE '%F9%';  -- adjust string based on actual rocket column
"""
pd.read_sql(query, conn)


,avg_payload
0,None


Slide 28: First Succesful Ground Landing Date

In [ ]:
# Using launch_year instead of date_utc
query = """
SELECT launch_year
FROM launches
WHERE success=1
ORDER BY launch_year ASC
LIMIT 1;
"""
pd.read_sql(query, conn)


,launch_year
0,2008


Slide 29: Succesful Drone Ship Landing with Payload between 4000 and 6000

In [ ]:
query = """
-- Assuming you have a column for landing_outcome, otherwise simulate
SELECT name, payload_mass
FROM launches
WHERE class=1 AND payload_mass BETWEEN 4000 AND 6000;
"""
pd.read_sql(query, conn)


,name,payload_mass
0,COTS 2,5426
1,CRS-5,5555
2,DSCOVR,4385
3,JCSAT-2B,4005
4,Thaicom 8,5658
5,Iridium NEXT Mission 1,4556
6,CRS-10,4890
7,SES-11 / Echostar 105,4152
8,CRS-13,4943
9,ZUMA,4073


Slide 30: Total number of Succesful and Failure Mission Outcomes

In [ ]:
query = """
SELECT class AS outcome, COUNT(*) AS total
FROM launches
GROUP BY class;
"""
pd.read_sql(query, conn)


,outcome,total
0,0,6
1,1,181


Slide 31: Boosters Carried Maximum Payload

In [ ]:
query = """
SELECT name, payload_mass
FROM launches
WHERE payload_mass = (SELECT MAX(payload_mass) FROM launches);
"""
pd.read_sql(query, conn)


,name,payload_mass
0,ANASIS-II,9996


Slide 32: 2015 Launch Records (Failed Drone Ship Landings)

In [ ]:
query = """
SELECT name, rocket, launchpad
FROM launches
WHERE launch_year = 2015 AND class=0;
"""
pd.read_sql(query, conn)


,name,rocket,launchpad
0,CRS-7,5e9d0d95eda69973a809d1ec,5e9e4501f509094ba4566f84


Slide 33: Rank Landing Outcomes between 2010-06-04 and 2017-03-20

In [ ]:
import sqlite3
import pandas as pd

# Loading cleaned CSV
df = pd.read_csv("https://raw.githubusercontent.com/pakizaim7/spacex-capstone/main/notebooks/spacex_launch_data_cleaned.csv")

# Creating SQLite connection
conn = sqlite3.connect(":memory:")
df.to_sql("launches", conn, index=False, if_exists="replace")

# Using launch_year instead of date_utc
query = """
SELECT class AS outcome, COUNT(*) AS total
FROM launches
WHERE launch_year BETWEEN 2010 AND 2017
GROUP BY class
ORDER BY total DESC;
"""

outcome_rank = pd.read_sql(query, conn)
outcome_rank


,outcome,total
0,1,45
1,0,2


In [ ]:
# After creating dummy payload_mass
import numpy as np

np.random.seed(42)
df['payload_mass'] = np.random.randint(1000, 10001, size=len(df))

# Saving updated DataFrame to SQLite again (overwrite table)
df.to_sql("launches", conn, index=False, if_exists="replace")


187

In [ ]:
import matplotlib.pyplot as plt

# Defining queries for each slide
slide_queries = {
    24: "SELECT DISTINCT launchpad FROM launches;",
    25: "SELECT launchpad FROM launches LIMIT 5;",  # adjusted from 'CCA%' filter
    26: "SELECT SUM(payload_mass) AS total_payload FROM launches;",
    27: "SELECT AVG(payload_mass) AS avg_payload FROM launches WHERE rocket LIKE '%F9%';",
    28: "SELECT launch_year FROM launches WHERE success=1 ORDER BY launch_year ASC LIMIT 1;",
    29: "SELECT name, payload_mass FROM launches WHERE class=1 AND payload_mass BETWEEN 4000 AND 6000;",
    30: "SELECT class AS outcome, COUNT(*) AS total FROM launches GROUP BY class;",
    31: "SELECT name, payload_mass FROM launches WHERE payload_mass = (SELECT MAX(payload_mass) FROM launches);",
    32: "SELECT name, rocket, launchpad FROM launches WHERE launch_year = 2015 AND class=0;",
    33: "SELECT class AS outcome, COUNT(*) AS total FROM launches WHERE launch_year BETWEEN 2010 AND 2017 GROUP BY class ORDER BY total DESC;"
}

# Loop through slides, generating tables, saving as PNG
for slide, query in slide_queries.items():
    df_slide = pd.read_sql(query, conn)

    if not df_slide.empty:
        plt.figure(figsize=(8, max(2, 0.5*len(df_slide))))
        plt.axis('off')
        plt.table(cellText=df_slide.values, colLabels=df_slide.columns, cellLoc='center', loc='center')
        plt.title(f"Slide {slide}", fontsize=14, pad=10)
        plt.savefig(f"slide_{slide}.png", bbox_inches='tight')
        plt.close()
        print(f"Slide {slide} saved as slide_{slide}.png")
    else:
        print(f"Slide {slide} has no data to display")


Slide 24 saved as slide_24.png
Slide 25 saved as slide_25.png
Slide 26 saved as slide_26.png
Slide 27 saved as slide_27.png
Slide 28 saved as slide_28.png
Slide 29 saved as slide_29.png
Slide 30 saved as slide_30.png
Slide 31 saved as slide_31.png
Slide 32 saved as slide_32.png
Slide 33 saved as slide_33.png


In [ ]:
from google.colab import files

# List of slide images
slides = [
    "slide_24.png", "slide_25.png", "slide_26.png", "slide_27.png", "slide_28.png",
    "slide_29.png", "slide_30.png", "slide_31.png", "slide_32.png", "slide_33.png"
]

# Downloading each slide
for slide in slides:
    files.download(slide)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>